In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
# from config import config
import os
import numpy as np


In [2]:
model_base = pd.read_csv(r"C:\Users\HP\Desktop\KOMAL\DATA\check.csv")


In [3]:
model_base["LeadID"]=model_base.index
model_base

,Gender,Age,Marital Status,Education,Employment,Area,family members,Income,Awarness about OTT,OTT user,...,Genre_Thriller,Genre_Drama,Genre_Romance,Genre_Action,Genre_Crime,Genre_Family,Genre_Sci-Fi,Genre_Horror,Total_Genre,LeadID
0,1,22,1,0,4,2,3,4,1,0,...,1,0,0,1,1,0,1,0,5,0
1,0,22,1,2,4,3,2,0,1,0,...,1,1,1,0,1,0,0,1,6,1
2,0,22,1,1,4,2,3,0,1,0,...,1,1,0,0,0,0,0,0,2,2
3,1,23,0,1,4,3,2,3,1,0,...,1,1,1,1,1,0,1,0,7,3
4,1,21,1,1,4,3,4,0,1,0,...,1,1,0,0,1,0,1,1,6,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,1,25,1,0,0,3,2,3,1,0,...,1,0,0,1,0,0,1,0,4,402
403,1,25,1,0,0,3,0,2,0,0,...,0,0,0,0,0,0,0,0,1,403
404,1,27,0,1,0,2,1,1,1,0,...,1,1,0,1,1,0,0,1,6,404
405,0,55,0,3,1,3,2,0,1,0,...,0,1,0,0,0,0,0,0,1,405


In [4]:
list(model_base.columns)

['Gender',
 'Age',
 'Marital Status',
 'Education',
 'Employment',
 'Area',
 'family members',
 'Income',
 'Awarness about OTT',
 'OTT user',
 'Platform for entertainment',
 'How did you know about OTT',
 'OTT over Traditional',
 'Subscription BL',
 'Duration of OTT use',
 'Yearly spend on OTT',
 'Regular watching ',
 'time spent BL',
 'time spent AL',
 'OTT vs tv',
 'Internet_Source_Wi-Fi',
 'Internet_Source_Mobile data',
 'Internet_Source_Hotspot',
 'Internet_Source_Dongle',
 'No_of_Internet_Source',
 'Language_English',
 'Language_Hindi',
 'Language_Marathi',
 'Language_South_Indian',
 'Language_Foreign',
 'Language_Gujurati',
 'Language_Other',
 'No_of_languages',
 'Device_TV',
 'Device_PC',
 'Device_Mob',
 'Device_stick',
 'Device_Other',
 'No_of_Device',
 'Purchased_TV',
 'Purchased_Stand',
 'Purchased_Headphones',
 'Purchased_internet',
 'Purchased_data_plans',
 'Purchased_Nothing',
 'No_of_Purchased',
 'Ideal_time_FreeTime',
 'Ideal_time_Travelling',
 'Ideal_time_Holidays',
 'I

In [5]:


if __name__ == "__main__":
    
    target_col = "OTT_Subscription_PRIME"

    print(model_base[target_col].value_counts())


    X_train, X_test, y_train, y_test = train_test_split(model_base, model_base[target_col],
                                                        test_size=0.3,
                                                        random_state=1, stratify=model_base[target_col])

    X_train.to_csv(r"C:\Users\HP\Desktop\KOMAL\OUTPUT_n\train_df.csv", index=False)
    print(X_train[target_col].value_counts())
    print(X_train.shape)
    X_test.to_csv(r"C:\Users\HP\Desktop\KOMAL\OUTPUT_n\test_df.csv", index=False)
    print(X_test[target_col].value_counts())

    print(X_test.shape)



1    219
0    188
Name: OTT_Subscription_PRIME, dtype: int64
1    153
0    131
Name: OTT_Subscription_PRIME, dtype: int64
(284, 102)
1    66
0    57
Name: OTT_Subscription_PRIME, dtype: int64
(123, 102)


In [6]:
!pip install xgboost

In [7]:
import xgboost as xgb

In [10]:
import pandas as pd
from datetime import datetime
import os
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
import xgboost as xgb
import pickle
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.utils import class_weight

import os
os.chdir(r"C:\Users\HP\Desktop\KOMAL\ANS")


def confusion_matrix(df, target_col, top_per=0.0):
    op_df = deepcopy(df)

    if top_per !=0:
        op_df.sort_values(["pred_prob_1"], ascending=False, inplace=True)
        op_df.reset_index(drop=True, inplace=True)
        top_len = int(top_per * op_df.shape[0])
        op_df["y_pred"] = 0
        op_df.loc[0:top_len, "y_pred"]=1

    pivot_df = pd.pivot_table(op_df, index=target_col, columns="y_pred", values="LeadID", aggfunc="count")
    pivot_df["Total"] = pivot_df.sum(axis=1)
    pivot_df.loc["Total", :] = pivot_df.sum(axis=0)
    pivot_df.loc["Precision", :]=[np.nan, np.nan, pivot_df.loc[1, 1]*100/pivot_df.loc["Total", 1]]
    pivot_df.loc["Recall", :]=[np.nan, np.nan, pivot_df.loc[1, 1]*100/pivot_df.loc[1, "Total"]]
    pivot_df.loc["Per_Base_Pred", :]=[np.nan, np.nan, pivot_df.loc["Total", 1]*100/pivot_df.loc["Total", "Total"]]

    pivot_df.reset_index(inplace=True)
    pivot_df["OTT_Subscription_PRIME"] = ["Actual_0", "Actual_1" ,"Total", "Precision", "Recall", "Per_Base_Pred"]
    pivot_df.rename(columns={0:"Pred_0", 1:"Pred_1"}, inplace=True)
    return pivot_df





if __name__ == '__main__':

    
    target_col = "OTT_Subscription_PRIME"

    #
    training_df = pd.read_csv(r"C:\Users\HP\Desktop\KOMAL\OUTPUT_n\train_df.csv")
    #
    # # read validation sample
    validation_df = pd.read_csv(r"C:\Users\HP\Desktop\KOMAL\OUTPUT_n\test_df.csv")




    non_calc_columns = [
    "LeadID",
        target_col]

   
    feature_list = [

"Yearly spend on OTT",
"OTT_Subscription_NETFLIX",
"Subscription BL",
"OTT_Subscription_HOTSTAR",
"No_of_Device",
"Internet_Source_Wi-Fi",
"Genre_Action",
"Genre_Sci-Fi",
"time spent AL",
"Content_Movies",
"OTT_Subscription_ZEE5",

 ]


    print("check")
    # feature_list.remove("LeadID")

    # training_df.fillna(0, inplace=True)
    # validation_df.fillna(0, inplace=True)

    print("number of features", len(feature_list))

    t_corr_df = training_df[feature_list].corr()
    t_corr_df.round(2).to_csv(r"t_corr.csv")

    t_vif = pd.DataFrame()
    t_vif["VIF Factor"] = [variance_inflation_factor(training_df[feature_list].values, i) for i in range(training_df[feature_list].shape[1])]
    t_vif["features"] = feature_list
    t_vif.to_csv(r"training_data_vif.csv", index=False)
    


    xg_clf = xgb.XGBClassifier(colsample_bytree=0.6, gamma=1, learning_rate=0.01, max_depth=8,
                               n_estimators=250, objective='binary:logistic',
                               subsample=0.7, reg_alpha=0.1, reg_lambda=2,
                               seed=123, max_delta_step=0, verbosity=3, n_jobs=3,
                               scale_pos_weight=0.5*np.sum(training_df[target_col]==0)/np.sum(training_df[target_col]==1))

    xg_clf.fit(training_df[feature_list], training_df[target_col])

    preds = xg_clf.predict(training_df[feature_list])
    # training_df["y_pred"] = preds
    # training_df["pred_prob"] = xg_clf.predict_proba(training_df[feature_list])[:,1]
    pred_prob = xg_clf.predict_proba(training_df[feature_list])

    prob_name_list = ["pred_prob_"+str(xg_clf.classes_[0]), "pred_prob_" + str(xg_clf.classes_[1])]

    training_df["pred_prob_"+str(xg_clf.classes_[0])] = pred_prob[:,0]
    training_df["pred_prob_" + str(xg_clf.classes_[1])] = pred_prob[:, 1]

    training_df["y_pred"] = preds

    # training_df.ix[0, "max_prob"] =  training_df["pred_prob"].max()
    # training_df.ix[0, "min_prob"] =  training_df["pred_prob"].min()
    # training_df.ix[0, "avg_prob"] =  training_df["pred_prob"].mean()
    # , "max_prob", "min_prob", "avg_prob"
    training_df[["LeadID", target_col,"y_pred"]+prob_name_list]\
        .round(2).to_csv(r"training_data_pred.csv", index=False)

    # training_df[["LeadID", target_col, "y_pred", "pred_prob"]].round(2)\
    #     .to_csv("D:/Analytics/Wayne/PMS_Redemption/xgboost_model_building/new_y/training_data_pred.csv", index=False)

    # # plot tree
    # xgb.plot_tree(xg_clf, num_trees=0)
    # plt.rcParams['figure.figsize'] = [50, 10]
    # plt.show()

    # feature importance df
    feat_imp_df = pd.DataFrame()
    feat_imp_df["Feature_Name"] = feature_list
    feat_imp_df["Imp"] = xg_clf.feature_importances_

    feat_imp_df.sort_values(["Imp"], ascending=False, inplace=True)
    feat_imp_df.reset_index(drop=True, inplace=True)
    # feat_imp_df.round(2).to_csv("D:/Analytics/Wayne/PMS_Redemption/xgboost_model_building/new_y/training_data_feat_imp.csv",
    #         index=False)

    feat_imp_df.to_csv(r"training_data_feat_imp.csv",
            index=False)

    # fscore df
    # xg_clf.get_booster().get_fscore()

    # # plot feature importance
    # xgb.plot_importance(xg_clf)
    # plt.tight_layout()
    # # plt.rcParams['figure.figsize'] = [5, 5]
    # plt.savefig("D:/Analytics/Wayne/PMS_Redemption/xgboost_model_building/new_y/training_data_feat_imp.png")
    # plt.close()

    # save the model to disk
    pickle.dump(xg_clf, open(r"xgboost_model.pkl", 'wb'))


    v_y_pred = xg_clf.predict(validation_df[feature_list])
    # validation_df["y_pred"] = v_y_pred
    v_pred_prob = xg_clf.predict_proba(validation_df[feature_list])

    validation_df["pred_prob_"+str(xg_clf.classes_[0])] = v_pred_prob[:,0]
    validation_df["pred_prob_" + str(xg_clf.classes_[1])] = v_pred_prob[:, 1]
    # validation_df["pred_prob_" + str(xg_clf.classes_[2])] = v_pred_prob[:, 2]

    validation_df["y_pred"] = v_y_pred

    # validation_df.ix[0, "max_prob"] = validation_df["pred_prob"].max()
    # validation_df.ix[0, "min_prob"] = validation_df["pred_prob"].min()
    # validation_df.ix[0, "avg_prob"] = validation_df["pred_prob"].mean()
    # "max_prob", "min_prob", "avg_prob"

    validation_df[["LeadID", target_col,"y_pred"] + prob_name_list].round(2)\
        .to_csv(r"validation_data_pred.csv", index=False)


    train_cf = confusion_matrix(training_df,target_col)
    

    val_cf = confusion_matrix(validation_df,target_col)
    

    train_cf.round(2).to_csv(r"train_cf.csv", index=False)
    

    val_cf.round(2).to_csv(r"val_cf.csv", index=False)
    





check
number of features 12
[21:09:47] ======== Monitor: Learner ========
[21:09:47] Configure: 0.000535s, 1 calls @ 535us

[21:09:47] ======== Monitor: GBTree ========
[21:09:47] ======== Monitor: TreePruner ========
[21:09:47] DEBUG: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/gbm/gbtree.cc:155: Using tree method: 2
[21:09:47] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 12 extra nodes, 8 pruned nodes, max_depth=4
[21:09:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:09:47] INFO: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/tree/updater_prune.cc:101: tree pruning end, 8 extra nodes, 4 pruned nodes, max_depth=3
[21:09:4